In [21]:
import os
import subprocess
import sys
import getpass
from pathlib import Path

# --- CONFIGURAÇÃO ---
raw_source = "./"
# Se o path começar com "/media~/", substitui "~" pelo seu usuário em /media
username = getpass.getuser()
if raw_source.startswith("/media/~/"):
    corrected = raw_source.replace("/media/~/", f"/media/{username}/")
else:
    corrected = raw_source
# Agora expande "~" se ainda existir
SOURCE_DIR = Path(os.path.expanduser(corrected))
OUTPUT_DIR = SOURCE_DIR / "pdfs"
PYTHON_EXE = sys.executable
# ---------------------

# Verifica pasta de origem
if not SOURCE_DIR.exists():
    raise FileNotFoundError(f"Pasta de origem não encontrada: {SOURCE_DIR}")

# Cria apenas a pasta pdfs dentro de SOURCE_DIR, sem recriar o resto
OUTPUT_DIR.mkdir(exist_ok=True)

# Converte cada .ipynb em PDF
for nb_path in SOURCE_DIR.glob("*.ipynb"):
    pdf_name = nb_path.stem + ".pdf"
    cmd = [
        PYTHON_EXE, "-m", "jupyter", "nbconvert",
        str(nb_path),
        "--to", "pdf",
        "--output", pdf_name,
        "--output-dir", str(OUTPUT_DIR),
        "--debug"
    ]
    print(f"🔄 Convertendo {nb_path.name} → {pdf_name}")
    try:
        out = subprocess.run(cmd, check=True, capture_output=True, text=True)
        print(out.stdout)
    except subprocess.CalledProcessError as e:
        print(f"❌ Erro ao converter {nb_path.name}:\n{e.stderr}")
print("✅ Conversão de todos os notebooks finalizada.")


🔄 Convertendo 1 - Data Loading.ipynb → 1 - Data Loading.pdf

🔄 Convertendo 2 - Data Analysis.ipynb → 2 - Data Analysis.pdf

🔄 Convertendo report.ipynb → report.pdf

✅ Conversão de todos os notebooks finalizada.


In [ ]:
# Analisando os documentos escreva uma apresentação detalhada do estudo, em alto nível.
# Escrever 15 parágrafo
# Não incluir nada de python
# Explicar em alto nivel
# Um leigo deve entender do que se trata




# Analisando os documentos descreva a metodologia e os resultados que foram obtidos no estudo.
# Escrever 15 parágrafo
# Não incluir nada de python
# Explicar em alto nivel
# Um leigo deve entender do que se trata




# Analisando os documentos descreva a analise e discussão e os resultados que foram obtidos no estudo.
# Escrever 15 parágrafo
# Não incluir nada de python
# Explicar em alto nivel
# Um leigo deve entender do que se trata




# Analisando os documentos descreva as conclusões do estudo
# Escrever 15 parágrafo
# Não incluir nada de python
# Explicar em alto nivel
# Um leigo deve entender do que se trata


# Agora com base nos texto gerado nas etapas anteriores quero um relatório completo.
# Este relatório deve comunicar os achados para os tomadores de decisão
# Não incluir nada de python
# Explicar em alto nivel
# Um leigo deve entender do que se trata
# Estrutura do relatório: Apresentação (4 paragrafos), metodologia e resultados(8 paragrafos), análise e discussão(4 paragrafos), conclusão(2 paragrafos)





In [ ]:
# Analisando os documentos extraia os codigos que geram visualizações.
# Adapte estes codigos para salvar as imagens em PNG na pasta "figuras"
# Nomear estas Figuras em ordem, figura_1.png, figura_2.png, figura_3.png, et
# Retornar um codigo unico que gere e salve todas as imangens em png na pasta correta

In [6]:
from src.libs.lib import *
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df_original = pd.read_csv("src/data/tabela_ocorrencias_dbpx_com_a_pontuação_academia.csv")


Hash numérica de 5 dígitos: 94799


In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

# Preparar o DataFrame
df = df_original.copy()

# Conversões necessárias
df['Pontuação'] = pd.to_numeric(df['Pontuação'], errors='coerce')
df['Pontuação Atualização'] = pd.to_numeric(df['Pontuação Atualização'], errors='coerce')
df['Pontuação Comitê'] = pd.to_numeric(df['Pontuação Comitê'], errors='coerce')
df['occurence_create'] = pd.to_datetime(df['occurence_create'], errors='coerce')

# Quartis da Pontuação Atualização
try:
    pontuacoes_validas = df['Pontuação Atualização'].dropna()
    bin_edges = pd.qcut(pontuacoes_validas, q=4, retbins=True, duplicates='drop')[1]
    label_count = len(bin_edges) - 1
    labels = [f"Q{i+1}" for i in range(label_count)]
    df['pontuacao_quartil'] = pd.qcut(df['Pontuação Atualização'], q=label_count, labels=labels, duplicates='drop')
except Exception as e:
    print("Erro ao criar quartis de pontuação:", e)
    df['pontuacao_quartil'] = None

# Quartis da coluna observation (se possível)
try:
    obs_numeric = pd.to_numeric(df['observation'], errors='coerce')
    df['observation_quartil'] = pd.qcut(obs_numeric, q=4, duplicates='drop')
except:
    df['observation_quartil'] = None

# Criar a pasta 'figuras' se necessário
os.makedirs("figuras", exist_ok=True)

# Função para gerar e salvar gráficos sequencialmente
def gerar_graficos_e_salvar(df):
    figura_id = 1

    def salvar_fig(titulo):
        nonlocal figura_id
        filename = f"figuras/figura_{figura_id}.png"
        plt.savefig(filename, bbox_inches='tight')
        print(f"{titulo} → {filename}")
        figura_id += 1
        plt.close()

    # Histograma
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    sns.histplot(df['Pontuação Atualização'].dropna(), kde=True, color='blue', bins=30)
    plt.title('Histograma - Pontuação Atualização')

    plt.subplot(1, 2, 2)
    sns.histplot(df['Pontuação Comitê'].dropna(), kde=True, color='green', bins=30)
    plt.title('Histograma - Pontuação Comitê')
    salvar_fig("Histogramas")

    # Boxplot
    plt.figure(figsize=(14, 8))
    sns.boxplot(data=df, x='Pontuação Atualização', y='description')
    plt.title('Boxplot - Pontuação Atualização por Tipo de Ocorrência')
    salvar_fig("Boxplot Pontuação por Ocorrência")

    # Frequência
    plt.figure(figsize=(12, 6))
    df['description'].value_counts().head(20).plot(kind='bar')
    plt.title('Frequência das Ocorrências (Top 20)')
    plt.xlabel('Descrição')
    plt.ylabel('Frequência')
    plt.xticks(rotation=75)
    salvar_fig("Frequência de Ocorrências")

    # Barras empilhadas
    if df['pontuacao_quartil'].notna().any():
        top10_desc = df['description'].value_counts().head(10).index
        temp = df[df['description'].isin(top10_desc)]
        crosstab = pd.crosstab(temp['description'], temp['pontuacao_quartil'], normalize='index')
        crosstab.plot(kind='bar', stacked=True, figsize=(12, 6), colormap='viridis')
        plt.title('Distribuição por Quartil nas Top 10 Ocorrências')
        plt.ylabel('Proporção')
        salvar_fig("Distribuição por Quartil")

    # Linha temporal
    df.set_index('occurence_create').resample('M')['Pontuação Atualização'].mean().plot(figsize=(12, 5))
    plt.title('Média da Pontuação Atualização por Mês')
    plt.ylabel('Média')
    plt.xlabel('Data')
    salvar_fig("Média por Mês")

    # Heatmap correlação
    plt.figure(figsize=(8, 6))
    sns.heatmap(df[['Pontuação', 'Pontuação Atualização', 'Pontuação Comitê']].corr(), annot=True, cmap='coolwarm')
    plt.title('Correlação entre as Pontuações')
    salvar_fig("Heatmap de Correlação")

    # Dispersão
    plt.figure(figsize=(8, 6))
    sns.scatterplot(x='Pontuação Atualização', y='Pontuação Comitê', data=df)
    plt.title('Dispersão: Atualização vs Comitê')
    salvar_fig("Dispersão Pontuação")

    # Heatmap de contingência
    if df['observation_quartil'].notna().any():
        cont_table = pd.crosstab(df['description'], df['observation_quartil'])
        plt.figure(figsize=(14, 8))
        sns.heatmap(cont_table, annot=False, cmap='Blues')
        plt.title('Heatmap: Descrição × Quartil de Observação')
        salvar_fig("Contingência Descrição x Observação")

    # Top 20 motoristas
    plt.figure(figsize=(12, 6))
    df['driver_id'].value_counts().head(20).plot(kind='bar')
    plt.title('Top 20 Motoristas com Mais Ocorrências')
    plt.xlabel('driver_id')
    plt.ylabel('Ocorrências')
    salvar_fig("Top Motoristas")

    # Cascata de impacto
    impacto = df.groupby('description')['Pontuação Atualização'].sum().sort_values(ascending=False).head(15)
    plt.figure(figsize=(14, 6))
    plt.bar(impacto.index, impacto.values)
    plt.xticks(rotation=90)
    plt.title('Impacto Total da Pontuação Atualização por Evento')
    salvar_fig("Cascata de Impacto")

# Rodar
gerar_graficos_e_salvar(df)


Histogramas → figuras/figura_1.png
Boxplot Pontuação por Ocorrência → figuras/figura_2.png
Frequência de Ocorrências → figuras/figura_3.png
Distribuição por Quartil → figuras/figura_4.png


/tmp/ipykernel_216134/2523262898.py:86: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  df.set_index('occurence_create').resample('M')['Pontuação Atualização'].mean().plot(figsize=(12, 5))


Média por Mês → figuras/figura_5.png
Heatmap de Correlação → figuras/figura_6.png
Dispersão Pontuação → figuras/figura_7.png
Contingência Descrição x Observação → figuras/figura_8.png
Top Motoristas → figuras/figura_9.png
Cascata de Impacto → figuras/figura_10.png


In [ ]:
# Analisando o relatório gerado na etapa anterior quero que gere um código python para gerar um fluxograma
# Incluir neste fluxograma as etapas do estudo
# Incluir os principais resultados
# Incluir os principais achados
# Incluir as principais conclusões
# O fluxograma deve ser salvo na pasta "figuras" como um arquivo png
# O nome do arquivo deve ser "fluxograma.png"
# O código deve ser único e gerar o fluxograma completo
# O código deve ser executável e gerar o fluxograma completo
# Código de referencia: from graphviz import Digraph# Fluxograma acadêmico estilizadoflow = Digraph('Academic_Flowchart', format='png')# Atributos do grafo (layout)flow.attr('graph',           rankdir='TB',           # Direção top-to-bottom para estilo acadêmico          fontsize='12',           fontname='Times New Roman',          bgcolor='white',           margin='0.2')# Atributos dos nós (estilo acadêmico)flow.attr('node',           shape='rectangle',           style='rounded,filled',           fillcolor='#F7F9FB',    # Tom suave          color='#2E4053',        # Borda escura          fontname='Times New Roman',           fontsize='12',           margin='0.2')# Atributos das arestas (linhas)flow.attr('edge',           color='#2E4053',           arrowhead='vee',           penwidth='1.5')# Definição das etapasetapas = [    ('A', 'Início'),    ('B', '1. Importação de Dados\n(pandas)'),    ('C', '2. Pré-processamento\n(numpy, unidecode)'),    ('D', '3. Estatística Descritiva\n(statsmodels)'),    ('E', '4. Modelagem Preditiva\n(scikit-learn)'),    ('F', '5. Visualização de Dados\n(matplotlib, seaborn)'),    ('G', '6. Exportação de Relatórios\n(openpyxl, tabulate)'),    ('H', '7. Deploy & BQ\n(functions-framework,\ngoogle-cloud-bigquery)'),    ('I', 'Conclusão')]# Cria os nósfor key, label in etapas:    flow.node(key, label)# Conecta as etapas em sequênciafor (src, _), (dst, _) in zip(etapas, etapas[1:]):    flow.edge(src, dst)# Exibe o fluxogramaflow

In [8]:
from graphviz import Digraph
import os

# Criar pasta se necessário
os.makedirs("figuras", exist_ok=True)

# Criar fluxograma
fluxograma = Digraph('Fluxograma_Estudio', format='png')

# Configurações de layout e estilo
fluxograma.attr('graph', rankdir='TB', fontsize='12', fontname='Times New Roman', bgcolor='white', margin='0.2')
fluxograma.attr('node', shape='rectangle', style='rounded,filled', fillcolor='#F7F9FB',
                color='#2E4053', fontname='Times New Roman', fontsize='12', margin='0.2')
fluxograma.attr('edge', color='#2E4053', arrowhead='vee', penwidth='1.5')

# Definição das etapas do estudo
etapas = [
    ('A', 'Início do Estudo'),
    ('B', '1. Coleta de Dados\n67.554 registros analisados'),
    ('C', '2. Organização dos Dados\nTipos de eventos e pontuações'),
    ('D', '3. Análise Exploratória\nFrequência, boxplots, dispersão, correlações'),
    ('E', '4. Identificação de Padrões\nExcesso de "Dia agenciado", reincidências'),
    ('F', '5. Análise Temporal\nVariação das médias mensais'),
    ('G', '6. Revisões e Ajustes\nDiferenças entre sistema e comitê'),
    ('H', '7. Resultados-Chave\n- 95% "Dia agenciado"\n- Inconsistências em revisões\n- Perfis reincidentes'),
    ('I', '8. Conclusões\nNecessidade de revisão de critérios,\ntratamento da reincidência e padronização'),
    ('J', 'Fim do Estudo')
]

# Criar nós
for key, label in etapas:
    fluxograma.node(key, label)

# Criar conexões
for i in range(len(etapas) - 1):
    src, _ = etapas[i]
    dst, _ = etapas[i + 1]
    fluxograma.edge(src, dst)

# Salvar o fluxograma
fluxograma.render(filename='fluxograma', directory='figuras', cleanup=True)


'figuras/fluxograma.png'

In [ ]:

# Agora com base no relatório completo quero que gere um documento HTML para ele. 
# Não usar Python nesta etapa.
# Gerar em HTMl puramente.
# Usar as normas da ABNT
# Neste documento incluir o texto adaptado do relatório. 
# Incluir também as figuras geradas na etapa anterior. com titulos e descrição.
# Incluir e citar no texto as figuras a partir dos PNGs gerados na etapa anterior
# Também incluir o fluxograma gerado na etapa anterior.
# Deixar responsivo
